(introduction-to-simphony)=
# Introduction to simphony

Before we start this tutorial, you should know the basics of Python. We expect you to have an Python environment set up,
with the simphony package installed.

Our goal with this tutorial is to define and simulate a simple circuit. In Simphony, circuits are represented all in a
single Python file. We'll go through the typical objects found in every circuit definition, in order.


## Models

Models are the basic components in simphony, which are used to represent an element in a photonic circuit. Each model has a 

The base Model class provides a common interface for all models, including methods to get and set the scattering 
parameters, get references to ports, rename ports, and check if a model is connected to any others. It also 
provides methods to convert the model to a scikit-rf network, which can be useful for further analysis and 
simulation.


```{eval-rst}
Here's an overview of the Model parent class :py:class:`simphony.models.Model`:

.. autoclass:: simphony.models.Model
    :noindex:
```

```{note}
A basic model has no ``__init__()`` function. It is only required if the model takes in parameters (width or length, for
example) that  affect the scattering parameters.
```

All models in Simphony extend this parent class, but redefine ports, and s-parameters to match the device
they represent.

## Instantiating Models

Before we can use a model in our circuit, we need to instantiate it. Simphony includes a library of models from the [SiEPIC PDK](https://github.com/SiEPIC) (developed at the University of British Columbia).
We might define a couple of models with the following:

In [1]:
from simphony.libraries import siepic

# waveguide of 2.5 mm length
wg1 = siepic.Waveguide(length=2500, loss=3)
# waveguide of 7.5 mm length
wg2 = siepic.Waveguide(length=7500, loss=3)

```{eval-rst}
These are both :py:class:`simphony.libraries.siepic.Waveguide` models. The siepic components are parameterizable, so we can pass different parameters when instantiating them. In this case ``wg1`` will be a shorter waveguide than ``wg2``.
Thus the two will have differing s-parameters for each model since each waveguide was set to have 3 dB of loss per cm.
```

```{note}
Convention in simphony is to use um for units of length.
```

## Creating a Circuit

```{eval-rst}
A circuit consists of at least two connected models. Models are connect via their :py:class:`simphony.models.OPort` class.
```

Using our previous two components to demonstrate, the simplest way to connect ports is as follows:

In [2]:
from simphony.circuit import Circuit

ckt = Circuit()
ckt.connect(wg1.o(1), wg2.o(0))

# Circuit s-parameters at 1550 nm
print(ckt.s_params(1.55))

[[[ 0.        +0.j        -0.53715961-0.4611364j]
  [-0.53715961-0.4611364j  0.        +0.j       ]]]


## Simulation

```{eval-rst}
A :py:class:`simphony.circuit.Circuit` class automatically perform a subnetwork growth algorithm to calculate the resultant s-parameters of the circuit based on the individual s-parameters of each model it is composed of. A :py:class:`simphony.circuit.Circuit` can then be passed to a simulator from :py:module:`simphony.simulation` which provides a collection of simulators. 

Let's run a simple wl sweep simulation on the circuit we have created using the :py:class:`simphony.simulation.classical.ClassicalSim` class.
``````

In [3]:
from simphony.simulation import ClassicalSim

# Create a simulation and add a laser and detector
sim = ClassicalSim(ckt, wl=1.55)
sim.add_laser(ckt.o(0))
sim.add_detector(ckt.o(1))

# Run the simulation
result = sim.run()

# Since the total wg length is 1 cm and the loss is 3 dB/cm, the power should be 50%.
print(f"Power transmission: {abs(result.output[0, 0])**2}")


Power transmission: 0.5011872336272725
